In [1]:
import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt
import cmath
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import mean_squared_error
import gym
from gym import spaces
from tqdm import tqdm

def read_excel(filepath,kind):
	excel_file = pd.ExcelFile(filepath)
	active_sheet = excel_file.sheet_names[0]
	df = pd.read_excel(filepath,sheet_name=active_sheet)
	headers = df.columns.tolist()
	if kind == 1: # 列
		columns_data = {}
		for header in headers:
			columns_data[header] = df[header].tolist()
		return columns_data
	elif kind == 0: # 行
		rows_data = []
		for index, row in df.iterrows():
			rows_data.append(row.tolist())
		columns_data = {}
		columns_data[headers[0]] = headers[1:]
		for lists in rows_data:
			columns_data[lists[0]] = lists[1:]
		return columns_data
df = read_excel('./HANDLE_data/ThetaData.xlsx',1)
DELTA_THETA = df['θ/θideal']
M = df['M']
OMEGA = df['w']
TR = df['Tr']
PR = df['pr']

In [2]:
X = np.column_stack((M, OMEGA, TR, PR))
y = np.array(DELTA_THETA)

# 划分
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 定义高斯过程回归模型
kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))
model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# 主动学习策略
def select_data(model, X_pool, n_samples=10):
    # 计算每个样本的不确定性（使用预测标准差作为不确定性指标）
    y_pred, sigma = model.predict(X_pool, return_std=True)
    uncertainties = sigma
    # 选择不确定性最高的样本
    selected_indices = np.argsort(uncertainties)[-n_samples:]
    return selected_indices

# 定义强化学习环境
class ActiveLearningEnv(gym.Env):
    def __init__(self, model, X_pool, y_pool, n_samples_per_iteration):
        super(ActiveLearningEnv, self).__init__()
        self.model = model
        self.X_pool = X_pool
        self.y_pool = y_pool
        self.n_samples_per_iteration = n_samples_per_iteration
        self.action_space = spaces.Discrete(len(X_pool))
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(X_pool),), dtype=np.float32)

    def reset(self):
        self.model.fit(X_train, y_train)
        y_pred, sigma = self.model.predict(self.X_pool, return_std=True)
        self.uncertainties = sigma
        return self.uncertainties

    def step(self, action):
        selected_indices = np.argsort(self.uncertainties)[-self.n_samples_per_iteration:]
        selected_X = self.X_pool[selected_indices]
        selected_y = self.y_pool[selected_indices]

        # 将选择的数据点添加到训练集中
        global X_train, y_train
        X_train = np.vstack((X_train, selected_X))
        y_train = np.hstack((y_train, selected_y))

        # 从池中移除选择的数据点
        mask = np.ones(len(self.X_pool), dtype=bool)
        mask[selected_indices] = False
        self.X_pool = self.X_pool[mask]
        self.y_pool = self.y_pool[mask]

        # 训练模型
        self.model.fit(X_train, y_train)

        # 计算奖励
        y_val_pred = self.model.predict(X_val)
        val_loss = mean_squared_error(y_val, y_val_pred)
        reward = -val_loss

        # 更新不确定性
        y_pred, sigma = self.model.predict(self.X_pool, return_std=True)
        self.uncertainties = sigma

        done = len(self.X_pool) == 0
        return self.uncertainties, reward, done, {}

# 创建强化学习环境
env = ActiveLearningEnv(model, X_test, y_test, n_samples_per_iteration=10)

# 定义强化学习代理
class RandomAgent:
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation):
        return self.action_space.sample()

agent = RandomAgent(env.action_space)

In [3]:
n_episodes = 10
for episode in tqdm(range(n_episodes)):
    observation = env.reset()
    done = False
    while not done:
        action = agent.act(observation)
        observation, reward, done, info = env.step(action)
        print(f'Episode {episode+1}/{n_episodes}, Reward: {reward:.10f}')

  0%|          | 0/10 [00:16<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
y_test_pred = model.predict(X_test)
test_loss = mean_squared_error(y_test, y_test_pred)
print(f'Test Loss: {test_loss:.10f}')

Test Loss: 0.0009573305


In [ ]:
import joblib
model_path = './MODEL/MODEL_3_NONETRUL.pkl'
joblib.dump(model, model_path)
print(f'Model saved to {model_path}')

Model saved to ./MODEL/MODEL_2_TRY_BYES.pth


In [ ]:
loaded_model = joblib.load(model_path)
print('Model loaded and ready for inference')

Model loaded and ready for inference
